In [10]:
# Formule
from math import pi as π
from dataclasses import dataclass
from rberga06.phylab import Datum as d, Measure

def V_cil(H: Measure[float], R: Measure[float]):
    """Volume del cilindro."""
    return π * H * R**2

def I_cil(density: Measure[float], H: Measure[float], R: Measure[float]):
    """Momento d'inerzia del cilindro rispetto all'asse centrale"""
    return π/2 * density * H * R**4

def V_tc(dH: Measure[float], R: Measure[float], r: Measure[float]):
    """Volume del tronco di cono."""
    return π/3 * dH * (R**3 - r**3)/(R - r)

def I_tc(density: Measure[float], dH: Measure[float], R: Measure[float], r: Measure[float]):
    """Momento d'inerzia del tronco di cono rispetto all'asse centrale"""
    return π/10 * density * dH * (R**5 - r**5)/(R - r)

@dataclass
class Cil:
    h: Measure[float]
    r: Measure[float]
    def V(self, /) -> Measure[float]: return V_cil(self.h, self.r)
    def I(self, density: Measure[float], /) -> Measure[float]: return I_cil(density, self.h, self.r)

@dataclass
class Tcn:
    h: Measure[float]
    R: Measure[float]
    r: Measure[float]
    def V(self, /) -> Measure[float]:
        return V_tc(self.h, self.R, self.r)
    def I(self, density: Measure[float], /) -> Measure[float]:
        return I_tc(density, self.h, self.R, self.r)


## Corpo che rotola

In [11]:
from typing import cast
from math import degrees, atan
from rberga06.phylab import Datum as d

h = [x/1000 for x in [
    d(30.45, .05),
    d( 5.95,  .1),
    d( 9.20,  .1),
    d(10.80, .05),
    d( 4.25, .05),
    d(52.95, .05),
    d( 4.25, .05),
    d(10.80, .05),
    d( 9.25,  .1),
    d( 5.95,  .1),
    d(30.40, .05),
]]

diam1 = [d(x, .05)/1000 for x in [
    49.85, 31.00, 25.85, 18.80, 35.85, 49.90, 35.95, 18.65, 25.90, 31.45, 49.95,
]]
diam2 = [d(x, .05)/1000 for x in [
    49.90, 29.40, 25.85, 18.65, 34.55, 49.90, 36.35, 18.75, 25.90, 29.10, 49.90,
]]

diam = diam2

obj = [
    Cil(h[0], diam[0]/2),
    Tcn(h[1], diam[0]/2, diam[1]/2),
    Cil(h[2], diam[2]/2),
    Cil(h[3], diam[3]/2),
    Tcn(h[4], diam[4]/2, diam[5]/2),
    Cil(h[5], diam[5]/2),
    Tcn(h[6], diam[5]/2, diam[6]/2),
    Cil(h[7], diam[7]/2),
    Cil(h[8], diam[8]/2),
    Tcn(h[9], diam[10]/2, diam[9]/2),
    Cil(h[10], diam[10]/2),
]

M = d(2214.57, .01)/1000  # kg
V = sum([x.V() for x in obj])
density = M / V
for _i, _x in enumerate(obj):
    print(f"I#{_i}", _x.I(density))
I = cast(Measure[float], sum([x.I(density) for x in obj], start=d(0., 0.)))
R = (diam[2] + diam[8])/4
D_ = (diam[0] + diam[5] + diam[-1])/3
K = I / (M * R**2) + 1

print("R =", R*1000, "mm")
print("density =", density, "kg/m^3")
print("V =", V, "m^3")
print("I =", I * 1e4, "10⁻⁴ kg m²")
print("I/(MR^2) + 1 =", K)
print(f"D_ = ", D_)
_mu_MR2_Ip1 = d(.3, .1) * ((M*R**2)/I + 1)
print("arctan[µ(MR^2/I + 1)] =", d(
    degrees(atan(_mu_MR2_Ip1.best)),
    degrees(_mu_MR2_Ip1.delta/(1+_mu_MR2_Ip1.best**2)),
), "\b°")

I#0 Datum(best=0.0001546407818563279, delta=1.8402685068972811e-06)
I#1 Datum(best=1.3666206112313878e-05, delta=4.643467506431115e-07)
I#2 Datum(best=3.3648412109271607e-06, delta=8.363897081586994e-08)
I#3 Datum(best=1.0702214655090365e-06, delta=2.3120762136760076e-08)
I#4 Datum(best=1.1799930238456128e-05, delta=3.7591025601344237e-07)
I#5 Datum(best=0.0002689073694348953, delta=3.0124429576775917e-06)
I#6 Datum(best=1.2636172891971497e-05, delta=4.2296734060602314e-07)
I#7 Datum(best=1.0933605530218792e-06, delta=2.3558118865716773e-08)
I#8 Datum(best=3.4093795071482746e-06, delta=8.449480830888986e-08)
I#9 Datum(best=1.3520553339953977e-05, delta=4.577837793033035e-07)
I#10 Datum(best=0.0001543868561061533, delta=1.837663674784436e-06)
R = Datum(best=12.937500000000002, delta=0.025) mm
density = Datum(best=8343.238585457932, delta=52.14713689635419) kg/m^3
V = Datum(best=0.000265432898426271, delta=1.657817351063042e-06) m^3
I = Datum(best=6.384956727166783, delta=0.0862619592605

$$g = \frac{2L}{t^2 \sin\theta}\left(\frac{I_\text{cm}}{M R^2} + 1\right)$$

In [12]:
from math import sin, cos, radians

angoli = [d(radians(x.best), radians(x.delta)) for x in [
    # unità di misura: °
    d(3.4, 0.1),
]]
distanze = [
    d(55.6, .1)/100,  # m
]

L = distanze[0] - d(1.6, .1)/100 - D_
t = 2_021_348.32/1_000_000  # s
g = 2 * K * L / (t ** 2 * d(sin(angoli[0].best), cos(angoli[0].best) * angoli[0].delta))
print("g =", g, "m/s²")

g = Datum(best=11.012966967875542, delta=0.4906930768404661) m/s²
